## Imports 

In [1]:
from statistics import mean
import numpy as np
import pandas as pd
import math
import os
from collections import Counter
from functools import reduce
import glob
import copy

## Opening the CSV files 

In [2]:
dataframes = [pd.read_csv(file, sep=',', index_col=0) for file in sorted(glob.glob('../preprocessed_datasets' + "/*."+'csv'))]
cohorts = [file.strip(".csv") for file in sorted(os.listdir('../preprocessed_datasets'))]

In [3]:
# reduce to BL visit only
all_cohorts = dict()
for name, df in zip(cohorts, dataframes):
    all_cohorts[name] = df.loc[(df["Visit"] == 1) & (df["Diagnosis"].astype(str) == 'AD')]

## Functions to perform essential calculations

In [4]:
def cat_stat_df(dfs, result):
    """Counting different categories, calculate the % of categorical features, store results in a df"""
    
    categorical = {'APOE4': [2.0, 1.0], 'Sex': ['Female'], 'Diagnosis': ['CU', 'MCI', 'AD']}
    column_cat = ['Sex', 'Diagnosis', 'APOE4']

    for cohort in dfs:
        
        if dfs[cohort].empty==True:
            continue
        
        else:
            calc_dict = dict()
            df = dfs[cohort]

            for col in column_cat:
                ca = Counter(df[col].dropna())
                calc_dict[col] = ca

            cohort_df = pd.DataFrame(calc_dict).transpose()
            cohort_df = cohort_df.dropna(how='all')
            cohort_df.loc[cohort] = cohort_df.sum()
           
            for i in categorical:
                
                if i == 'Diagnosis':
                    
                    if i in cohort_df.index: 
                        result.loc[cohort, categorical[i]] = cohort_df.loc[cohort, cohort_df.loc[i].notna()].astype(int)
                        result.loc[cohort, categorical[i]] = result.loc[cohort, categorical[i]].replace({np.nan: 0})
                        result.loc[cohort, 'n'] = int(sum(cohort_df.loc[cohort, cohort_df.loc[i].notna()]))
                        result.loc[cohort, 'Total'] = int(len(dfs[cohort].index))
                    
                    else:
                        result.loc[cohort, i] = np.nan
                        result.loc[cohort, 'n'] = int(len(dfs[cohort].index))
                
                elif i == 'APOE4':
                    
                    if 'APOE4' in list(cohort_df.index.astype(str)):
                        
                        if '2.0' not in list(cohort_df.columns.astype(str)) and '2' not in list(cohort_df.columns.astype(str)):
                            cohort_df[2.0] = np.nan
                        
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) / 
                                                     sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    
                    else:
                        result.loc[cohort, i] = np.nan
                
                elif i == 'Sex':
                    
                    if (i in cohort_df.index) & ("Female" in cohort_df.columns):
                        result.loc[cohort, i] = round(100 * sum([val for val in cohort_df.loc[i, categorical[i]]]) 
                                                      / sum([val for val in cohort_df.loc[i].dropna()]), 1)
                    else:
                        result.loc[cohort, i] = 0
    
                    
    result.rename(columns={"Sex": "Female %", "APOE4": "APOE4 %"}, inplace=True)
              
    return result

In [5]:
def num_stat_df(dfs, result_df):
    """Calculating std and mean and storing it in the result dataframe"""
    
    column_names = ['Age', 'CDR', 'Education', 'MMSE', 'CDRSB', 'Hippocampus', 'A-beta', 'Ttau', 'Ptau']
    
    for df in dfs:
        dataset = dfs[df]
        calc_dict = dict()
        
        for col in column_names:
            
            if (col in dataset.columns) and (dataset[col].notna().any()):
                df_std = round(np.nanstd(dataset[col]), 1)
                df_mean = round(np.nanmean(dataset[col]), 1)
                dict_value = str(df_mean) + ' (' + str(df_std) + ')'
                calc_dict[col] = dict_value
                
            else:
                calc_dict[col] = np.nan
   
        for key in calc_dict:
            result_df.loc[df, key] = calc_dict[key]
        
    return result_df

## Make an empty dataframe to fill in with the results

In [6]:
results = pd.DataFrame(index = all_cohorts.keys(), columns = [col for col in all_cohorts['AIBL'].columns])
results.index.name = 'Name of Dataset'

for i in ['CU', 'MCI', 'AD', 'Total']:
    results[i] = np.nan
    
cat_stat_df(all_cohorts, results)
num_stat_df(all_cohorts, results)

results.drop(columns=['Diagnosis', 'Visit', 'Race', 'Months'], inplace=True)
results

,Age,Female %,Education,APOE4 %,CDR,MMSE,CDRSB,Hippocampus,Ttau,Ptau,A-beta,CU,MCI,AD,Total,n
Name of Dataset,,,,,,,,,,,,,,,,
A4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABVIB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADNI,74.9 (7.8),43.7,15.2 (2.9),66.7,0.8 (0.3),23.2 (2.1),4.4 (1.7),5761.9 (1021.4),366.6 (145.5),36.5 (15.8),667.4 (313.9),0.0,0.0,389.0,389.0,389.0
AIBL,78.8 (8.3),63.5,12.2 (2.7),61.9,1.0 (0.5),19.0 (5.5),5.7 (3.1),2.5 (0.4),NaN,NaN,NaN,0.0,0.0,181.0,181.0,181.0
ANM,78.6 (7.1),62.9,9.4 (4.3),54.3,1.1 (0.6),20.2 (5.3),6.6 (3.8),5235.8 (934.1),NaN,NaN,NaN,0.0,0.0,512.0,512.0,512.0
ARWIBO,73.9 (8.5),66.9,7.4 (4.2),41.2,1.0 (0.4),20.5 (5.0),NaN,5429.1 (1043.9),647.7 (313.7),86.7 (41.6),363.3 (145.9),0.0,0.0,281.0,281.0,281.0
DOD-ADNI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EDSD,72.7 (8.0),57.6,10.7 (3.6),57.1,NaN,20.9 (5.2),NaN,5676.2 (1174.4),1150.0 (0.0),156.0 (0.0),634.0 (0.0),0.0,0.0,151.0,151.0,151.0
EMIF,70.4 (8.7),46.8,10.3 (4.1),NaN,0.8 (0.3),21.7 (4.7),NaN,6224.8 (929.1),479.5 (437.4),70.2 (43.1),399.1 (162.3),0.0,0.0,201.0,201.0,201.0


## Final table 

In [7]:
results[['n', 'Total', 'CU', 'MCI', 'AD', 'Female %', 'Age', 'Education', 'MMSE', 'CDR', 'CDRSB', 'APOE4 %', 'Hippocampus']]

,n,Total,CU,MCI,AD,Female %,Age,Education,MMSE,CDR,CDRSB,APOE4 %,Hippocampus
Name of Dataset,,,,,,,,,,,,,
A4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ABVIB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ADNI,389.0,389.0,0.0,0.0,389.0,43.7,74.9 (7.8),15.2 (2.9),23.2 (2.1),0.8 (0.3),4.4 (1.7),66.7,5761.9 (1021.4)
AIBL,181.0,181.0,0.0,0.0,181.0,63.5,78.8 (8.3),12.2 (2.7),19.0 (5.5),1.0 (0.5),5.7 (3.1),61.9,2.5 (0.4)
ANM,512.0,512.0,0.0,0.0,512.0,62.9,78.6 (7.1),9.4 (4.3),20.2 (5.3),1.1 (0.6),6.6 (3.8),54.3,5235.8 (934.1)
ARWIBO,281.0,281.0,0.0,0.0,281.0,66.9,73.9 (8.5),7.4 (4.2),20.5 (5.0),1.0 (0.4),NaN,41.2,5429.1 (1043.9)
DOD-ADNI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
EDSD,151.0,151.0,0.0,0.0,151.0,57.6,72.7 (8.0),10.7 (3.6),20.9 (5.2),NaN,NaN,57.1,5676.2 (1174.4)
EMIF,201.0,201.0,0.0,0.0,201.0,46.8,70.4 (8.7),10.3 (4.1),21.7 (4.7),0.8 (0.3),NaN,NaN,6224.8 (929.1)
